In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# Read the CSV and Perform Basic Data Cleaning

In [2]:
# Load the data
file = "CountyFoodAccess.csv"
df = pd.read_csv(file)
# check data type
df.dtypes 

state_county           object
Urban                 float64
Pop2010                 int64
OHU2010                 int64
MedianFamilyIncome    float64
LA1and10                int64
LIPct                 float64
dtype: object

In [3]:
# Find null values
for column in df.columns :
    print(f"Column {column} has {df[column].isnull().sum()} null values")

Column state_county has 0 null values
Column Urban has 0 null values
Column Pop2010 has 0 null values
Column OHU2010 has 0 null values
Column MedianFamilyIncome has 2 null values
Column LA1and10 has 0 null values
Column LIPct has 0 null values


In [4]:
# Remove null rows
#--Median Family Income has 748 null values which is 1.04%. This figure is insignificant compared to the importance of the
#feature towards prediction. So we deciide to drop null values than remove the column.
df=df.dropna()

In [5]:
#Find duplicate entries
print(f"Duplicate entries: {df.duplicated().sum()}")

Duplicate entries: 0


# Define Model

In [6]:
# Create features
X = df.drop(columns=['state_county','MedianFamilyIncome',"LA1and10"]) 

# Create target
y = df["LA1and10"]

# Split model
X_train, X_test, y_train, y_test = train_test_split(X,
    y, random_state=1)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


### Logistic Regression

In [7]:
# Define model
model = LogisticRegression(solver='lbfgs', random_state=1)
# fit model

# Fit model
model.fit(X_train, y_train)
# make predictions on the entire training dataset
predictions = model.predict(X_test)

# Evaluate the model
y_pred = model.predict(X_test)
print(f" Logistic regression model accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Logistic regression model accuracy: 0.581


In [8]:
cm_logreg = confusion_matrix(y_test,y_pred)
cm_logreg

array([[455,   2],
       [327,   1]])

In [9]:
#Run Logistic Regression on scaled data
# Fit model
model.fit(X_train_scaled, y_train)
# make predictions on the entire training dataset
predictions_scaled = model.predict(X_test_scaled)

# Evaluate the model
y_pred_scaled = model.predict(X_test_scaled)
print(f" Logistic regression model accuracy: {accuracy_score(y_test,y_pred_scaled):.3f}")

 Logistic regression model accuracy: 0.632


In [10]:
confusion_matrix(y_test,y_pred_scaled)

array([[339, 118],
       [171, 157]])

In [11]:
pd=pd.DataFrame({"Prediction": predictions, "Scaled Prediction":predictions_scaled, "Actual": y_test})
pd.head()

,Prediction,Scaled Prediction,Actual
3028,0,0,0
2257,0,1,0
2565,0,0,0
1876,0,0,0
45,0,0,1


### Undersampling

In [12]:
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler

In [13]:
Counter(y)

Counter({1: 1363, 0: 1777})

In [14]:
rus = RandomUnderSampler(random_state=1)
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({0: 1035, 1: 1035})

In [15]:
# Retry Logistic Regression with undersampled data
model.fit(X_resampled, y_resampled)
y_pred_resamp = model.predict(X_test)
print(f" Resampled logistic regression model accuracy: {accuracy_score(y_test,y_pred_resamp):.3f}")
confusion_matrix(y_test, y_pred_resamp)

 Resampled logistic regression model accuracy: 0.582


array([[416,  41],
       [287,  41]])

### SVM

In [16]:
# Create the SVM model
svm = SVC(kernel='linear')

# Train the model
svm.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred_svm = svm.predict(X_test_scaled)
print(f" SVM model accuracy: {accuracy_score(y_test,y_pred_svm):.3f}")

confusion_matrix(y_test, y_pred_svm)

 SVM model accuracy: 0.628


array([[318, 139],
       [153, 175]])

In [17]:
# Create the SVM model
svm_rbf = SVC()

# Train the model
svm_rbf.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred_svmrbf = svm_rbf.predict(X_test_scaled)
print(f" SVM model accuracy: {accuracy_score(y_test,y_pred_svmrbf):.3f}")

confusion_matrix(y_test, y_pred_svmrbf)

 SVM model accuracy: 0.661


array([[371,  86],
       [180, 148]])

### Random Forest

In [18]:
# Create Random Forest Classifier model
rf_model = RandomForestClassifier(n_estimators=100, random_state=78) 

In [19]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)
# Making predictions using the testing data.
rf_y_pred = rf_model.predict(X_test_scaled)
# Evaluate the model
print(f"Random Forest model accuracy: {accuracy_score(y_test,rf_y_pred):.3f}")

Random Forest model accuracy: 0.646


In [20]:
confusion_matrix(y_test,rf_y_pred)

array([[328, 129],
       [149, 179]])

In [21]:
rf_model.feature_importances_


array([0.27035658, 0.24893944, 0.26013095, 0.22057303])

### Neural Network

In [22]:
import tensorflow as tf

In [28]:
# Define neural network model
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=12, activation="relu", input_dim=4))
nn_model.add(tf.keras.layers.Dense(units=6, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=150)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/150
74/74 [==============================] - 1s 1ms/step - loss: 0.6942 - accuracy: 0.5737
Epoch 2/150
74/74 [==============================] - 0s 1ms/step - loss: 0.6728 - accuracy: 0.5902
Epoch 3/150
74/74 [==============================] - 0s 1ms/step - loss: 0.6645 - accuracy: 0.6403
Epoch 4/150
74/74 [==============================] - 0s 1ms/step - loss: 0.6591 - accuracy: 0.6505
Epoch 5/150
74/74 [==============================] - 0s 1ms/step - loss: 0.6547 - accuracy: 0.6573
Epoch 6/150
74/74 [==============================] - 0s 1ms/step - loss: 0.6502 - accuracy: 0.6620
Epoch 7/150
74/74 [==============================] - 0s 1ms/step - loss: 0.6466 - accuracy: 0.6603
Epoch 8/150
74/74 [==============================] - 0s 1ms/step - loss: 0.6431 - accuracy: 0.6616
Epoch 9/150
74/74 [==============================] - 0s 1ms/step - loss: 0.6400 - accuracy: 0.6607
Epoch 10/150
74/74 [==============================] - 0s 1ms/step - loss: 0.6379 - accuracy: 0.6611
Epoch 11/

74/74 [==============================] - 0s 1ms/step - loss: 0.5842 - accuracy: 0.6811
Epoch 84/150
74/74 [==============================] - 0s 1ms/step - loss: 0.5832 - accuracy: 0.6913
Epoch 85/150
74/74 [==============================] - 0s 1ms/step - loss: 0.5827 - accuracy: 0.6900
Epoch 86/150
74/74 [==============================] - 0s 1ms/step - loss: 0.5810 - accuracy: 0.6947
Epoch 87/150
74/74 [==============================] - 0s 1ms/step - loss: 0.5829 - accuracy: 0.6887
Epoch 88/150
74/74 [==============================] - 0s 1ms/step - loss: 0.5810 - accuracy: 0.6849
Epoch 89/150
74/74 [==============================] - 0s 1ms/step - loss: 0.5820 - accuracy: 0.6989
Epoch 90/150
74/74 [==============================] - 0s 1ms/step - loss: 0.5813 - accuracy: 0.6917
Epoch 91/150
74/74 [==============================] - 0s 1ms/step - loss: 0.5807 - accuracy: 0.6943
Epoch 92/150
74/74 [==============================] - 0s 1ms/step - loss: 0.5806 - accuracy: 0.6934
Epoch 93/150


In [24]:
print(f'Logistic Regression: {accuracy_score(y_test,y_pred):.3f}')
print(f'Scaled Logistic Regression: {accuracy_score(y_test,y_pred_scaled):.3f}')
print(f'Undersampled Logistic Regression: {accuracy_score(y_test,y_pred_resamp):.3f}')
print(f'SVM Accuracy: {accuracy_score(y_test,y_pred_svm):.3f}')

Logistic Regression: 0.581
Scaled Logistic Regression: 0.632
Undersampled Logistic Regression: 0.582
SVM Accuracy: 0.628
